In [15]:
import json
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import TweetTokenizer


In [16]:
data = pd.read_excel('./dataset.xlsx')

In [1]:
wnl = WordNetLemmatizer()
USdict = enchant.Dict("en_US")
stoplist = set(stopwords.words('english'))

# 获取处理后的英文推特
def get_tweets(filepath):
    tweets = []
    file = open(filepath, 'r')

    # 去除链接
    def remove_urls(vTEXT):
        vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
        return (vTEXT)

    # 获取文本
    for line in file.readlines():
        dic = json.loads(line)
        text = str(dic.get('text', '!'))
        if judge_english(text):
            tweets.append(remove_urls(text))

    return tweets


def get_word(text):
    text = text.replace('.', ' ')
    tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
    rawwords = tknzr.tokenize(text)
    words = [word.lower() for word in rawwords if USdict.check(word) and word.isalpha() and len(word) > 2]
    return words


# 词性还原
def get_pos_word(words):
    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return None

    words = pos_tag(words)

    pos_word = [wnl.lemmatize(tag[0], pos=get_wordnet_pos(tag[1]) or wordnet.NOUN) for tag in words]

    # 停用词过滤
    cleanwords = [word for word in pos_word if word not in stoplist]

    return cleanwords


if __name__ == '__main__':
    tweets = get_tweets(data)

    df = pd.DataFrame()
    df['tweets'] = tweets
    # 分词
    df['tweets'] = df['tweets'].apply(get_word)
    # 词形还原
    df['tweets'] = df['tweets'].apply(get_pos_word)
    # 删除tweets中的空列表
    df = df[~(df['tweets'].str.len() == 0)]
    # 将列表转换为字符串
    df['tweets'] = df['tweets'].apply(lambda x: ' '.join(x))
    # 保存文本
    df.to_(storename, encoding='utf-8')


NameError: name 'WordNetLemmatizer' is not defined